# AM207 Final Project

Paper: *Learned Uncertainty-Aware (LUNA) Bases for Bayesian Regression using Multi-Headed Auxiliary Networks*

207Notes:
 -  reproduce figure 6 (rows are random restarts, see main paper) with NLM 2 hidden layers 50-50
 -  new code: plot the priors
 -  verify in paper, how many iterations were done for this figure, verify data generating process
 - 
 -  

In [ ]:
from autograd import numpy as np
from autograd import grad
from autograd.misc.optimizers import adam, sgd
from autograd import scipy as sp
import autograd.numpy.random as npr
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import sys
import time

# our libraries
import utils
from nlm import NLM
from feed_forward import Feedforward
import bayes_helpers as bh

#config
from config import *

### Generate Data

In [ ]:

# training data
x_train, y_train, x_test = utils.generate_data()

# for model evaluation: computing log likelihood
x_valid, y_valid, x_test_not_used = utils.generate_data()



### Run Vanilla Nueral Network

In [ ]:
nn = Feedforward(nlm_architecture, random=random_seed)

t0 = time.time()
nn.fit(x_train, y_train, opt_params)
nn_time = np.round(time.time() - t0, 3)

mse = nn.objective_trace.min()

print(f"\n {nn_time} Seconds, Log Likilihood {mse}")

#predict on the test x-values
y_test_pred = nn.forward(nn.weights, x_test)

#visualize the function learned by the neural network
plt.scatter(x_train.flatten(), y_train.flatten(), color='black', label='data')
plt.plot(x_test.flatten(), y_test_pred.flatten(), color='red', label='learned neural network function')
plt.legend(loc='best')
plt.show()


### NLM Demo

In [ ]:
# test
test_nlm = NLM(prior_variance,
               y_noise_variance,
               regularization_param_nlm, 
               nlm_architecture, 
               random_state = random_seed)

In [ ]:

t0 = time.time()
test_nlm.train(x_train,y_train, opt_params)
nlm_time = np.round(time.time() - t0, 3)
nlm_mse = test_nlm.ff.objective_trace.min()

print(f"{nlm_time} Seconds, {nlm_mse} MSE")

In [ ]:
for predictive_name, boolean in {"prior NLM":True, "posterior NLM":False}.items():
    predictives, predictive_samples = test_nlm.predict(x_test,prior = boolean)
    bh.viz_pp_samples(x_train, y_train,x_test.flatten(),predictive_samples,predictive_name)

In [ ]:
from autograd import scipy as sp
import math
        
test_nlm.get_log_l(x_train,y_train,x_valid,y_valid)

## testing log likilihood

In [ ]:
# we should VERIFY that log likelihood at 5000 is greater than 1000
i_params = params.copy()
for i in [10,100,200,300,400,1000]:
    i_params['max_iteration'] = i
    test_nlm.train(x_train,y_train, i_params)
    print(test_nlm.get_log_l(x_train,y_train,x_valid,y_valid))

In [ ]:
# looks good
print(params['max_iteration'])
i_architecture = architecture.copy()
for i in [1,2,5,10,50]:
    print(f"\n width: {i}")
    i_architecture['width'] = i
    test_nlm_i = NLM(prior_var,y_var, regularization_param_nlm,i_architecture, random_state = np.random.RandomState(0))
    test_nlm_i.train(x_train,y_train, params)
    print(test_nlm_i.get_log_l(x_train,y_train,x_valid,y_valid))